# Code Interpreter Agent

### Connect to the Foundry

In [1]:
from util import get_project_client
project = get_project_client()

### Create the Agent with CodeInterpreter Tool

In [2]:
import azure.ai.agents as agents
from azure.ai.agents.models import FilePurpose

file = project.agents.files.upload_and_poll(
    file_path="data/sample.csv", 
    purpose=FilePurpose.AGENTS
)

In [3]:
from azure.ai.agents.models import CodeInterpreterTool
import os

code_interpreter = CodeInterpreterTool(file_ids=[file.id])
agent = project.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="my-codeinterpreter-agent",
    instructions="You are helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)


### Util method message handeling

In [4]:
from util import sendMessage

sendMessage(project, agent.id, "What is the EPS of 3M India Ltd")

Created thread, ID: thread_iv2A2nstH7GzqHmReHKR4CRq
Created message, ID: msg_CaWeUM7X8aqa839tIncoREbH
Run finished with status: RunStatus.COMPLETED
User: What is the EPS of 3M India Ltd
Assistant: To answer your query regarding the "EPS of 3M India Ltd", I'll need to examine the file you uploaded. Let me review its contents to find the relevant financial data.

I'll start by inspecting the file and searching for the Earnings Per Share (EPS) for 3M India Ltd.
Assistant: The **EPS (Earnings Per Share) of 3M India Ltd.** reported in your file is:

- **EPS:** ₹129.7
- **EPS TTM (Trailing Twelve Months):** ₹475.7

If you need more details or explanation, please let me know!


In [ ]:
sendMessage("Each file contains a form. Extract the name of the person from each form. Output the result in a table with two columns: file name and name of the person. The file names should be the same as the original file names.")

In [ ]:
sendMessage("Also include the number of stocks owned by each person.")